In [554]:
import pandas as pd
import numpy as np
import requests
import regex as re
import geopandas, shapely
import json
from functools import reduce
import operator
from pymongo import MongoClient,GEOSPHERE

In [562]:
# Set connection with MongoDB
conn = MongoClient("localhost:27017")
db = conn.get_database("final_project")
p = db.get_collection("places")
h = db.get_collection("homes")

In [3]:
p.create_index([("geometry",GEOSPHERE)])
h.create_index([("geometry",GEOSPHERE)])

'geometry_2dsphere'

In [555]:
def insert_object(df, collection):
    '''
    This function inserts all the information from the dataframe we created with the previous function as a Mongodb object.
    This object will have two field: title (name of the film or show) and reviews (array with all the reviews users have made)
    '''

    collection.insert_many(df.to_dict('records'))

    return "Data succesfully added"

### Getting home data and cleaning it

In [6]:
# Download JSON file 
data = json.load(open("data_vivienda.json", 'r', encoding='utf-8'))

In [7]:
data_vivienda = pd.DataFrame(data)
data_vivienda.columns

Index(['propertyCode', 'thumbnail', 'externalReference', 'numPhotos', 'floor',
       'price', 'propertyType', 'operation', 'size', 'exterior', 'rooms',
       'bathrooms', 'address', 'province', 'municipality', 'district',
       'country', 'neighborhood', 'latitude', 'longitude', 'showAddress',
       'url', 'hasVideo', 'status', 'newDevelopment', 'hasLift', 'priceByArea',
       'detailedType', 'suggestedTexts', 'hasPlan', 'has3DTour', 'has360',
       'parkingSpace', 'newDevelopmentFinished'],
      dtype='object')

In [8]:
data_vivienda = data_vivienda.drop(["thumbnail","propertyCode", "externalReference", "numPhotos", "hasVideo", "priceByArea", "detailedType", "country", "suggestedTexts", "hasPlan", "has3DTour", "has360", "newDevelopmentFinished"], axis = 1)

In [9]:
# I'm only interested in those houses inside Madrid, so I'm going to delete all observations that belong to another municipality
data_vivienda = data_vivienda[data_vivienda["municipality"]=="Madrid"]

In [10]:
data_vivienda.dtypes

floor              object
price             float64
propertyType       object
operation          object
size              float64
exterior             bool
rooms               int64
bathrooms           int64
address            object
province           object
municipality       object
district           object
neighborhood       object
latitude          float64
longitude         float64
showAddress          bool
url                object
status             object
newDevelopment       bool
hasLift            object
parkingSpace       object
dtype: object

In [60]:
data_vivienda.isnull().sum()
# Parkingspace has too many null values, and as we cannot fill the with a proper value the best option is to drop the whole column (explain in API the reason for the lack of information)
# Let's check the floor, status and hasLift variables

floor              108
price                0
propertyType         0
operation            0
size                 0
exterior             0
rooms                0
bathrooms            0
address              0
province             0
municipality         0
district             0
neighborhood         0
latitude             0
longitude            0
showAddress          0
url                  0
status             556
newDevelopment       0
hasLift            154
parkingSpace      6018
dtype: int64

In [11]:
data_vivienda = data_vivienda.drop("parkingSpace", axis = 1)

In [74]:
data_vivienda.status.value_counts()
# In this case we are going to fill the null values with 'unspecified'

good              8000
renew               24
newdevelopment       8
Name: status, dtype: int64

In [12]:
# There a couple of values that are not numbers, so I'm going to change them to a value that can be valid. In this case '0', as after looking a Idelista we can assume they are street level homes
data_vivienda.floor = data_vivienda.floor.replace(["bj","en","st","ss"], "0", regex = True)

In [13]:
data_vivienda.status = data_vivienda.status.fillna('unspecified')

In [14]:
data_vivienda.hasLift = data_vivienda.hasLift.fillna("unspecified")

In [15]:
data_vivienda = data_vivienda.dropna(how = 'any')

In [556]:
def geo_frame(df):
    df = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.latitude, df.longitude))
    df['geometry']=df['geometry'].apply(lambda x:shapely.geometry.mapping(x))
    return df

In [17]:
data_vivienda = geo_frame(data_vivienda)

c:\users\juanp\anaconda3\envs\ironhack\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [548]:
# As I have limited resources, and the number of queries I can do in all APIs is limited, I'm going to focus on three district with the more number of houses available

test = data_vivienda[data_vivienda.groupby('district').district.transform('count')>800]

In [549]:
test.district.value_counts()

Centro       1741
Salamanca    1727
Chamberí      972
Chamartín     813
Name: district, dtype: int64

In [414]:
insert_object(test, h)

'Data succesfully added'

In [874]:
test.to_csv("data\\test.csv")

### Google API

In [18]:
my_key = open("keys\\google_key.txt").readlines()[0]

In [7]:
response = requests.get(url)

#### Find places with google API

In [19]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

In [56]:
location = '40.446067,-3.691247' # I have picked Nuevos Minesterios as center of all queries
radius = '10000'
tipo = "restaurant"
page_token = my_key
url = f"{base_url}location={location}&radius={radius}&type={tipo}&key={my_key}"

In [57]:
response = requests.get(url)

In [59]:
prueba = response.json()

In [63]:
next_page_token = prueba["next_page_token"]
url_next = f"{base_url}pagetoken={next_page_token}&key={my_key}"

### FOURSQUARE API

In [623]:
client_id = open("keys\\foursquare_key.txt").readlines()[0]
client_secret = open("keys\\foursquare_key.txt").readlines()[1]

In [557]:

def extract(url):
    
    results = requests.get(url)

    code = json.loads(results.text)

    decoding = code.get("response")

    decoded = decoding.get("groups")[0]

    return decoded.get("items")

In [558]:
def getFromDict(diccionario,mapa):
    return reduce(operator.getitem,mapa,diccionario)

def places_df(frame,category):
    
    nombre = ["venue","name"]
    latitud = ["venue", "location","lat"]
    longitud = ["venue","location","lng"]
    barrio = ["venue", "location", "neighborhood"]
    tipo = ["venue", "categories", "pluralName"]
    x = []
    
    for diccionario in frame:
        lista = {}
        lista["name"] = getFromDict(diccionario,nombre)
        lista["longitude"] = getFromDict(diccionario,longitud)
        lista["latitude"] = getFromDict(diccionario,latitud)

        
        x.append(lista)
        
        df = pd.DataFrame(x)

    c = [category for _ in range(len(frame))]
        
    df["category"] = c
    
    return df

In [559]:
def locations(radius, limit,location, cat_id, category ):
    url = f'https://api.foursquare.com/v2/venues/explore?&client_id={client_id}&client_secret={client_secret}&v={version}&limit=200&ll={location}&radius={radius}&limit={limit}&categoryId={cat_id}'
    x = extract(url)
    return places_df(x, category)

### Find establishments using Foursquare to populate database

In [323]:
# IDs
nightlife = "4d4b7105d754a06376d81259"
gym = "4bf58dd8d48988d175941735"
park = "4bf58dd8d48988d163941735"
hospital = "4bf58dd8d48988d196941735"
elementary_school = "4f4533804b9074f6e4fb0105"
high_school = "4bf58dd8d48988d13d941735"
middle_school = "4f4533814b9074f6e4fb0106"
clothing_store = "4bf58dd8d48988d103951735"
daycare = "4f4532974b9074f6e4fb0104"
grocery_store = "4bf58dd8d48988d118951735"
supermarket = "52f2ab2ebcbc57f1066b8b46"
pharmacy = "4bf58dd8d48988d10f951735"
general_entertainment = "4d4b7104d754a06370d81259"
bar = "4bf58dd8d48988d116941735"
metro_station = "4bf58dd8d48988d1fd931735"
train_station = "4bf58dd8d48988d129951735"

#### Centro District

In [327]:
# I choose the centre of district Centro
location = "40.412951, -3.707111"

In [328]:
hospital_df_centro = locations(5000, 100, location, hospital, "medical_centre" )

In [330]:
gym_centro = locations(5000, 100, location, gym, "gym" )

In [333]:
ent_centro = locations(5000, 100, location, general_entertainment, "general_entertainment")

In [335]:
supermarkets_centro = locations(5000, 100, location, supermarket, "supermarket")

In [338]:
pharmacy_centro = locations(5000, 100, location, pharmacy, "medical_centre")

In [340]:
clothes_centro = locations(5000, 100, location, clothing_store, "clothing_store")

In [342]:
sub_centro = locations(5000, 100, location, metro_station, "transport")

In [343]:
train_centro = locations(5000, 100, location, train_station, "transport")

In [358]:
# There are somre results from train_centro that we are not interested in, so we can delete them. (Ex: Vía 17)
train_centro = train_centro.sort_values(by="name")

In [377]:
train_centro = train_centro[(train_centro["name"].str.contains("Cercanías")) | train_centro["name"].str.contains("Intercambiador") | train_centro["name"].str.contains("Estación")]


In [378]:
train_centro = train_centro.drop(train_centro[(train_centro["name"].str.contains("Andén")) | train_centro["name"].str.contains("Vía") | train_centro["name"].str.contains("AVE")].index)

In [348]:
nightlife_centro = locations(5000, 100, location, nightlife, "nightlife")

#### Salamanca District

In [494]:
location = "40.432751, -3.683338"

In [386]:
hospital_sal = locations(5000, 100, location, hospital, "medical_centre" )

In [388]:
gym_sal = locations(5000, 100, location, gym, "gym" )

In [390]:
ent_sal = locations(5000, 100, location, general_entertainment, "general_entertainment")

In [394]:
supermarkets_sal = locations(5000, 100, location, supermarket, "supermarket")

In [396]:
pharmacy_sal = locations(5000, 100, location, pharmacy, "medical_centre")

In [402]:
clothes_sal = locations(5000, 100, location, clothing_store, "clothing_store")

In [398]:
sub_sal = locations(5000, 100, location, metro_station, "transport")

In [399]:
train_sal = locations(5000, 100, location, train_station, "transport")

In [445]:
train_sal = train_sal.sort_values(by="name")

In [447]:
train_sal = train_sal[(train_sal["name"].str.contains("Cercanías")) | train_sal["name"].str.contains("Intercambiador") | train_sal["name"].str.contains("Estación")]

In [449]:
train_sal = train_sal.drop(train_sal[(train_sal["name"].str.contains("Andén")) | train_sal["name"].str.contains("Vía")| train_sal["name"].str.contains("AVE")].index)

In [495]:
nightlife_sal = locations(5000, 100, location, nightlife, "nightlife")

#### Chamartín Distric

In [496]:
location = "40.462781, -3.686737"

In [417]:
hospital_chamar = locations(5000, 100, location, hospital, "medical_centre" )

In [420]:
gym_chamar = locations(5000, 100, location, gym, "gym" )

In [422]:
ent_chamar = locations(5000, 100, location, general_entertainment, "general_entertainment")

In [425]:
supermarkets_chamar = locations(5000, 100, location, supermarket, "supermarket")

In [427]:
pharmacy_chamar = locations(5000, 100, location, pharmacy, "medical_centre")

In [434]:
clothes_chamar = locations(5000, 100, location, clothing_store, "clothing_store")

In [429]:
sub_chamar = locations(5000, 100, location, metro_station, "transport")

In [432]:
train_chamar = locations(5000, 100, location, train_station, "transport")

In [455]:
train_chamar = train_chamar[(train_chamar["name"].str.contains("Cercanías")) | train_chamar["name"].str.contains("Intercambiador") | train_chamar["name"].str.contains("Estación")]

In [459]:
train_chamar = train_chamar.drop(train_chamar[(train_chamar["name"].str.contains("Vip"))].index)

In [497]:
nightlife_chamar = locations(5000, 100, location, nightlife, "nightlife")

### Let's check if there are any duplicates in the extracted data

#### Subway and train stations

In [527]:
# Let's check if there are any duplicates in the subways stations
subs = pd.concat([sub_centro, sub_chamar, sub_sal, train_centro, train_sal, train_chamar])

In [528]:
subs.shape

(337, 4)

In [529]:
#Almost half of the values are dupkicated, so we proceed to eliminate them
subs.duplicated(subset = "name").sum()

165

In [530]:
subs = subs.drop_duplicates(subset="name", keep = 'first')

#### Hospitals and Pharmacies

In [533]:
hosp = pd.concat([hospital_df_centro, hospital_sal, hospital_chamar, pharmacy_centro, pharmacy_sal, pharmacy_chamar])

In [534]:
hosp.shape

(314, 4)

In [536]:
hosp.duplicated(subset="name").sum()

172

In [537]:
hosp = hosp.drop_duplicates(subset="name", keep='first')

#### Nightlife

In [517]:
night = pd.concat([nightlife_centro, nightlife_sal, nightlife_chamar])

In [518]:
night.shape

(300, 4)

In [520]:
night.duplicated(subset="name").sum()

103

In [521]:
night = night.drop_duplicates(subset="name", keep='first')

#### General Entertainment

In [522]:
general = pd.concat([ent_centro, ent_sal, ent_chamar])

In [523]:
general.shape

(299, 4)

In [525]:
general.duplicated(subset="name").sum()

139

In [526]:
general = general.drop_duplicates(subset="name", keep='first')

#### Gyms + clothes_shops and supermarkets
Many of this will surely have the same name, so we have to check duplicates using coordinates instead of name

In [500]:
gyms = pd.concat([gym_centro, gym_sal, gym_chamar])

In [501]:
gyms.duplicated(subset="name").sum()

172

In [502]:
pf = [clothes_centro, clothes_chamar, clothes_sal, gym_centro, gym_sal, gym_chamar, supermarkets_centro, supermarkets_sal, supermarkets_chamar ]

In [503]:
conjunto = pd.concat(pf)

In [508]:
conjunto = conjunto.sort_values(by = "name")

In [511]:
conjunto.duplicated(subset="longitude").sum()

466

In [512]:
conjunto = conjunto.drop_duplicates(subset="longitude", keep='first')

In [538]:
total = pd.concat([conjunto, night, gyms, hosp, general, subs, ])

In [ ]:
total = geo_frame(total)

In [541]:
insert_object(total, p)

'Data succesfully added'

In [157]:
# Gyms
gym_df = locations(9000, 100, gym, "gym")

In [153]:
# Museos
ent_df = locations(9000, 100, general_entertainment, "general_entertainment")

In [146]:
pharmacy_df = locations(9000, 100, pharmacy, "medical_centre")

In [159]:
supermarkets_df = locations(9000, 100, supermarket, "supermarket")

In [166]:
clothes_stores = locations(9000, 100 , clothing_store, "clothing_store")

In [281]:
sub_station = locations(11000, 100, metro_station, "transport")

In [304]:
train_station = locations(11000, 100, train_station, "transport")

In [313]:
nightlife = locations(10000, 100, nightlife, "nightlife")

In [170]:
conjunto = pd.concat([ent_df, gyms_df, hospitals_df, pharmacy_df, supermarkets_df, clothes_stores])

In [ ]:
conjunto = geo_frame(conjunto)

In [821]:
nightlife_sal

,name,longitude,latitude,category
0,Simposium,-3.681068,40.429714,nightlife
1,La Raquetista,-3.677165,40.420463,nightlife
2,Macera Tallerbar,-3.698176,40.426102,nightlife
3,Laredo,-3.675159,40.420313,nightlife
4,Bee Beer,-3.696900,40.421952,nightlife
...,...,...,...,...
95,Bodegas Ángel Sierra,-3.697528,40.422874,nightlife
96,Casa Camacho,-3.703880,40.425610,nightlife
97,La Mi Venta,-3.712599,40.420490,nightlife
98,Pinkleton & Wine,-3.709050,40.415390,nightlife


In [259]:
insert_object(conjunto, p)

'Data succesfully added'

In [182]:
conjunto.shape

(543, 5)